# Phase 3: Data Cleaning & Preprocessing

**Objective**: Clean and preprocess NHANES data for modeling with robust validation.

## Contents
1. [Load Data & Baseline Statistics](#1-load-data)
2. [Recode Special Values](#2-recode-special)
3. [Create Missing Flags](#3-missing-flags)
4. [Imputation](#4-imputation)
5. [Validation Checks](#5-validation)
6. [Save Outputs](#6-save-outputs)
7. [Summary](#7-summary)

In [1]:
# Standard imports
import sys
import json
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Project imports
from src.data.cleaners import (
    recode_special_values,
    create_missing_flags,
    impute_low_missing,
    impute_all_features,
    clean_pipeline,
    TARGET_RELATED_COLS
)
from src.data.validators import (
    check_ranges,
    check_logical_consistency,
    check_target_leakage,
    compare_distributions,
    generate_cleaning_report,
    validate_cleaned_data,
    load_config
)

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

# Load configuration
config = load_config()
print(f"Project: {config['project']['name']}")

Project: nhanes-diabetes-prediction


---
## 1. Load Data & Baseline Statistics <a name="1-load-data"></a>

In [2]:
# Load study population from Phase 2
input_path = project_root / 'data' / 'interim' / 'study_population.parquet'
df_raw = pd.read_parquet(input_path)

print(f"Loaded study population: {df_raw.shape}")
print(f"  Rows: {len(df_raw):,}")
print(f"  Columns: {len(df_raw.columns)}")

Loaded study population: (11723, 832)
  Rows: 11,723
  Columns: 832


In [3]:
# Baseline missing value statistics
missing_before = df_raw.isna().sum()
missing_pct_before = (missing_before / len(df_raw) * 100).round(2)

print("\n=== BASELINE MISSING VALUE SUMMARY ===")
print(f"Total missing values: {missing_before.sum():,}")
print(f"\nMissing value distribution:")
print(f"  Variables with 0% missing: {(missing_pct_before == 0).sum()}")
print(f"  Variables with <5% missing: {((missing_pct_before > 0) & (missing_pct_before < 5)).sum()}")
print(f"  Variables with 5-30% missing: {((missing_pct_before >= 5) & (missing_pct_before < 30)).sum()}")
print(f"  Variables with 30-50% missing: {((missing_pct_before >= 30) & (missing_pct_before < 50)).sum()}")
print(f"  Variables with 50%+ missing: {(missing_pct_before >= 50).sum()}")


=== BASELINE MISSING VALUE SUMMARY ===
Total missing values: 5,507,699

Missing value distribution:
  Variables with 0% missing: 61
  Variables with <5% missing: 38
  Variables with 5-30% missing: 209
  Variables with 30-50% missing: 43
  Variables with 50%+ missing: 481


In [4]:
# Document target-related columns (will NOT be imputed)
print("\nTarget-related columns (will NOT be imputed):")
for col in TARGET_RELATED_COLS:
    if col in df_raw.columns:
        missing = df_raw[col].isna().sum()
        pct = missing / len(df_raw) * 100
        print(f"  {col}: {missing:,} missing ({pct:.1f}%)")


Target-related columns (will NOT be imputed):
  DIQ070: 8,639 missing (73.7%)
  LBXGLU: 6,663 missing (56.8%)
  LBXGH: 1,128 missing (9.6%)
  DIABETES_STATUS: 25 missing (0.2%)
  DIQ010: 0 missing (0.0%)
  DIQ050: 0 missing (0.0%)


---
## 2. Recode Special Values <a name="2-recode-special"></a>

NHANES uses special codes:
- **7, 77, 777, etc.** = "Refused" → Recode to **-7**
- **9, 99, 999, etc.** = "Don't know" → Recode to **-9**

These are preserved as distinct categories rather than treated as missing.

In [5]:
# Check for special values before recoding
from src.data.cleaners import REFUSED_PATTERNS, DONT_KNOW_PATTERNS

print("Checking for NHANES special values...")
print(f"\nRefused patterns to recode: {REFUSED_PATTERNS}")
print(f"Don't know patterns to recode: {DONT_KNOW_PATTERNS}")

# Count special values in sample of columns
sample_cols = ['DIQ010', 'DIQ050', 'DIQ070', 'BPQ020', 'SMQ020', 'ALQ130']
existing_cols = [c for c in sample_cols if c in df_raw.columns]

print("\nSpecial values in questionnaire columns (before recoding):")
for col in existing_cols:
    refused = df_raw[col].isin(REFUSED_PATTERNS).sum()
    dont_know = df_raw[col].isin(DONT_KNOW_PATTERNS).sum()
    if refused > 0 or dont_know > 0:
        print(f"  {col}: Refused={refused}, Don't know={dont_know}")

Checking for NHANES special values...

Refused patterns to recode: {7777, 77777, 7, 777, 77}
Don't know patterns to recode: {99999, 99, 999, 9, 9999}

Special values in questionnaire columns (before recoding):
  DIQ010: Refused=0, Don't know=8
  DIQ050: Refused=1, Don't know=3
  DIQ070: Refused=1, Don't know=3
  BPQ020: Refused=0, Don't know=16
  SMQ020: Refused=2, Don't know=9
  ALQ130: Refused=52, Don't know=24


In [6]:
# Recode special values
df_recoded, recode_log = recode_special_values(df_raw, refused_code=-7, dont_know_code=-9)

print("\nRecoding summary:")
print(f"  Columns with 'Refused' recoded: {len(recode_log['refused'])}")
print(f"  Columns with 'Don't know' recoded: {len(recode_log['dont_know'])}")

2026-01-29 13:40:05,717 - INFO - Recoded 21364 'Refused' values to -7


2026-01-29 13:40:05,718 - INFO - Recoded 33388 'Don't know' values to -9



Recoding summary:
  Columns with 'Refused' recoded: 256
  Columns with 'Don't know' recoded: 364


In [7]:
# Verify recoding worked
print("\nVerifying recoding (sample columns):")
for col in existing_cols:
    n_refused = (df_recoded[col] == -7).sum()
    n_dont_know = (df_recoded[col] == -9).sum()
    if n_refused > 0 or n_dont_know > 0:
        print(f"  {col}: -7 (Refused)={n_refused}, -9 (Don't know)={n_dont_know}")


Verifying recoding (sample columns):
  DIQ010: -7 (Refused)=0, -9 (Don't know)=8
  DIQ050: -7 (Refused)=1, -9 (Don't know)=3
  DIQ070: -7 (Refused)=1, -9 (Don't know)=3
  BPQ020: -7 (Refused)=0, -9 (Don't know)=16
  SMQ020: -7 (Refused)=2, -9 (Don't know)=9
  ALQ130: -7 (Refused)=52, -9 (Don't know)=24


---
## 3. Create Missing Flags <a name="3-missing-flags"></a>

For features with >= 5% missing, create `_MISSING` indicator columns.
These flags can capture informative missingness patterns.

In [8]:
# Create missing flags for features with >= 5% missing
MISSING_FLAG_THRESHOLD = 0.05

df_with_flags, flag_columns = create_missing_flags(
    df_recoded, 
    threshold=MISSING_FLAG_THRESHOLD,
    exclude_cols=list(TARGET_RELATED_COLS)
)

print(f"\nCreated {len(flag_columns)} missing flag columns")
print(f"\nFirst 20 flag columns:")
for col in flag_columns[:20]:
    original_col = col.replace('_MISSING', '')
    missing_rate = df_recoded[original_col].isna().mean() * 100
    print(f"  {col}: {missing_rate:.1f}% missing")

/Users/timhall/Library/Mobile Documents/com~apple~CloudDocs/Projects/diabetes_prediction_project/src/data/cleaners.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[flag_col] = df[col].isna().astype(int)
/Users/timhall/Library/Mobile Documents/com~apple~CloudDocs/Projects/diabetes_prediction_project/src/data/cleaners.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[flag_col] = df[col].isna().astype(int)
/Users/timhall/Library/Mobile Documents/com~apple~CloudDocs/Projects/diabetes_prediction_project/src/data/cleane


Created 730 missing flag columns

First 20 flag columns:
  RIDAGEMN_MISSING: 100.0% missing
  RIDEXAGM_MISSING: 95.6% missing
  DMQADFC_MISSING: 90.7% missing
  DMDYRSUS_MISSING: 68.0% missing
  DMDEDUC3_MISSING: 95.2% missing
  RIDEXPRG_MISSING: 80.6% missing
  MIALANG_MISSING: 12.5% missing
  MIAPROXY_MISSING: 12.5% missing
  MIAINTRP_MISSING: 12.5% missing
  AIALANGA_MISSING: 29.1% missing
  DMDHRAGE_MISSING: 49.5% missing
  DMDHRBR4_MISSING: 51.6% missing
  DMDHREDU_MISSING: 51.6% missing
  DMDHRMAR_MISSING: 49.8% missing
  DMDHSEDU_MISSING: 75.3% missing
  INDFMPIR_MISSING: 12.8% missing
  BMXWT_MISSING: 6.3% missing
  BMIWT_MISSING: 95.9% missing
  BMXRECUM_MISSING: 100.0% missing
  BMIRECUM_MISSING: 100.0% missing


In [9]:
# Shape after adding flags
print(f"\nShape before flags: {df_recoded.shape}")
print(f"Shape after flags: {df_with_flags.shape}")
print(f"New columns added: {len(df_with_flags.columns) - len(df_recoded.columns)}")


Shape before flags: (11723, 832)
Shape after flags: (11723, 1562)
New columns added: 730


---
## 4. Imputation <a name="4-imputation"></a>

We create two versions:
1. **Minimal imputation** (for LightGBM): Only impute <5% missing, leave rest as NaN
2. **Full imputation** (for LogReg/NN): Impute all features

In [10]:
# Version 1: Minimal imputation for tree-based models
IMPUTE_THRESHOLD = 0.05

df_minimal, minimal_impute_log = impute_low_missing(
    df_with_flags,
    threshold=IMPUTE_THRESHOLD,
    target_cols=list(TARGET_RELATED_COLS)
)

print(f"\n=== MINIMAL IMPUTATION (for LightGBM) ===")
print(f"Columns imputed: {len(minimal_impute_log)}")
print(f"Missing values before: {df_with_flags.isna().sum().sum():,}")
print(f"Missing values after: {df_minimal.isna().sum().sum():,}")

2026-01-29 13:40:05,914 - INFO - Imputed 37 columns with <5% missing



=== MINIMAL IMPUTATION (for LightGBM) ===
Columns imputed: 37


Missing values before: 5,507,699
Missing values after: 5,491,041


In [11]:
# Show what was imputed (minimal)
print("\nColumns imputed (minimal strategy):")
for col, info in list(minimal_impute_log.items())[:15]:
    print(f"  {col}: {info['n_imputed']} values, impute_value={info['impute_value']:.2f}")


Columns imputed (minimal strategy):
  RIDEXMON: 580 values, impute_value=2.00
  DMDCITZN: 4 values, impute_value=1.00
  DMDEDUC2: 560 values, impute_value=4.00
  DMDMARTL: 560 values, impute_value=2.00
  FIALANG: 546 values, impute_value=1.00
  FIAPROXY: 546 values, impute_value=2.00
  FIAINTRP: 546 values, impute_value=2.00
  INDHHIN2: 573 values, impute_value=6.00
  INDFMIN2: 546 values, impute_value=6.00
  BMDSTATS: 580 values, impute_value=1.00
  DBQ229: 560 values, impute_value=2.00
  DBD905: 10 values, impute_value=0.00
  DBD910: 10 values, impute_value=0.00
  PAD680: 22 values, impute_value=300.00
  SLD012: 79 values, impute_value=6.50


In [12]:
# Version 2: Full imputation for linear/NN models
# Only impute features with <= 50% missing (features with >50% missing are too unreliable)
MAX_MISSING_RATE = 0.50

df_full, full_impute_log = impute_all_features(
    df_with_flags,
    target_cols=list(TARGET_RELATED_COLS),
    max_missing_rate=MAX_MISSING_RATE
)

print(f"\n=== FULL IMPUTATION (for LogReg/NN) ===")
print(f"Max missing rate for imputation: {MAX_MISSING_RATE*100:.0f}%")
print(f"Columns imputed: {len(full_impute_log)}")
print(f"Missing values before: {df_with_flags.isna().sum().sum():,}")
print(f"Missing values after: {df_full.isna().sum().sum():,}")

# Count columns skipped due to high missing rate
high_missing_cols = [col for col in df_with_flags.columns 
                     if df_with_flags[col].isna().mean() > MAX_MISSING_RATE
                     and col not in TARGET_RELATED_COLS
                     and not col.endswith('_MISSING')
                     and col not in ['SEQN', 'SURVEY_YEAR']]
print(f"Columns skipped (>{MAX_MISSING_RATE*100:.0f}% missing): {len(high_missing_cols)}")

2026-01-29 13:40:06,035 - INFO - Imputed 288 columns (missing rate <= 50%)


2026-01-29 13:40:06,035 - INFO - Skipped 479 columns with >50% missing (too unreliable)



=== FULL IMPUTATION (for LogReg/NN) ===
Max missing rate for imputation: 50%
Columns imputed: 288


Missing values before: 5,507,699
Missing values after: 4,963,349


Columns skipped (>50% missing): 479

In [13]:
# Compare the two versions
print("\n=== COMPARISON ===")
print(f"{'Metric':<35} {'Minimal':>15} {'Full':>15}")
print("-" * 65)
print(f"{'Columns imputed':<35} {len(minimal_impute_log):>15,} {len(full_impute_log):>15,}")
print(f"{'Remaining missing values':<35} {df_minimal.isna().sum().sum():>15,} {df_full.isna().sum().sum():>15,}")
print(f"{'Columns with any NaN':<35} {(df_minimal.isna().sum() > 0).sum():>15,} {(df_full.isna().sum() > 0).sum():>15,}")


=== COMPARISON ===
Metric                                      Minimal            Full
-----------------------------------------------------------------
Columns imputed                                  37             288


Remaining missing values                  5,491,041       4,963,349
Columns with any NaN                            734             483


In [14]:
# Verify target columns were NOT imputed
print("\nVerifying target columns were NOT imputed:")
for col in TARGET_RELATED_COLS:
    if col in df_raw.columns:
        before = df_raw[col].isna().sum()
        after_minimal = df_minimal[col].isna().sum()
        after_full = df_full[col].isna().sum()
        status = "OK" if before == after_minimal == after_full else "ERROR"
        print(f"  {col}: before={before}, minimal={after_minimal}, full={after_full} [{status}]")


Verifying target columns were NOT imputed:
  DIQ070: before=8639, minimal=8639, full=8639 [OK]
  LBXGLU: before=6663, minimal=6663, full=6663 [OK]
  LBXGH: before=1128, minimal=1128, full=1128 [OK]
  DIABETES_STATUS: before=25, minimal=25, full=25 [OK]
  DIQ010: before=0, minimal=0, full=0 [OK]
  DIQ050: before=0, minimal=0, full=0 [OK]


---
## 5. Validation Checks <a name="5-validation"></a>

In [15]:
# Load validation config
range_config = config.get('validation', {}).get('ranges', {})
print(f"Range checks configured for {len(range_config)} variables")

Range checks configured for 16 variables


In [16]:
# Run range checks on minimal imputed data
print("\n=== RANGE VALIDATION ===")
range_results = check_ranges(df_minimal, range_config)
range_results

2026-01-29 13:40:06,199 - WARNING - Found 215 values out of expected ranges


2026-01-29 13:40:06,200 - WARNING -   BMXBMI: 2 out of range


2026-01-29 13:40:06,201 - WARNING -   LBXGLU: 1 out of range


2026-01-29 13:40:06,201 - WARNING -   BPXDI1: 70 out of range


2026-01-29 13:40:06,201 - WARNING -   BPXDI2: 61 out of range


...

2026-01-29 13:40:06,201 - WARNING -   LBXTC: 2 out of range


2026-01-29 13:40:06,201 - WARNING -   LBDHDD: 7 out of range


2026-01-29 13:40:06,201 - WARNING -   LBXTR: 10 out of range



=== RANGE VALIDATION ===


,variable,min_expected,max_expected,actual_min,actual_max,n_below_min,n_above_max,n_out_of_range,pct_out_of_range
0,RIDAGEYR,18,80,18.0,80.0,0,0,0,0.00
1,BMXBMI,10,80,14.2,86.2,0,2,2,0.02
2,BMXWT,20,300,32.4,242.6,0,0,0,0.00
3,BMXHT,100,220,129.7,202.7,0,0,0,0.00
4,BMXWAIST,40,200,56.4,171.6,0,0,0,0.00
5,LBXGH,3,20,3.8,17.0,0,0,0,0.00
6,LBXGLU,30,500,21.0,479.0,1,0,1,0.02
7,BPXSY1,50,250,72.0,236.0,0,0,0,0.00
8,BPXSY2,50,250,72.0,238.0,0,0,0,0.00
9,BPXSY3,50,250,72.0,238.0,0,0,0,0.00


In [17]:
# Run logical consistency checks
print("\n=== LOGICAL CONSISTENCY CHECKS ===")
logic_results = check_logical_consistency(df_minimal)
logic_results

2026-01-29 13:40:06,210 - WARNING - Found 697 logical consistency violations


2026-01-29 13:40:06,211 - WARNING -   current_weight_le_max_weight: 683 violations


2026-01-29 13:40:06,211 - WARNING -   waist_lt_height: 9 violations


2026-01-29 13:40:06,211 - WARNING -   diabetes_meds_implies_diabetes: 5 violations



=== LOGICAL CONSISTENCY CHECKS ===


,check_name,description,n_violations,pct_violations,example_seqns
0,current_weight_le_max_weight,Current weight should be <= max weight ever,683,5.826,"[83742.0, 83743.0, 83788.0, 83875.0, 83889.0]"
1,waist_lt_height,Waist circumference should be < height,9,0.077,"[84396.0, 89652.0, 89810.0, 92059.0, 95058.0]"
2,diastolic_lt_systolic_1,Diastolic BP1 should be < Systolic BP1,0,0.000,[]
3,diastolic_lt_systolic_2,Diastolic BP2 should be < Systolic BP2,0,0.000,[]
4,diastolic_lt_systolic_3,Diastolic BP3 should be < Systolic BP3,0,0.000,[]
5,diabetes_meds_implies_diabetes,"If taking diabetes medication, should report h...",5,0.043,"[84332.0, 92981.0, 93353.0, 101449.0, 102517.0]"


In [18]:
# Check for target leakage
print("\n=== TARGET LEAKAGE CHECK ===")

# Define potential feature columns (exclude targets and identifiers)
feature_cols = [col for col in df_minimal.columns 
                if col not in TARGET_RELATED_COLS 
                and col not in ['SEQN', 'SURVEY_YEAR']]

leaked_cols = check_target_leakage(feature_cols, list(TARGET_RELATED_COLS))

if leaked_cols:
    print(f"WARNING: Found {len(leaked_cols)} leaked columns: {leaked_cols}")
else:
    print("No target leakage detected in feature columns")

2026-01-29 13:40:06,216 - INFO - No target leakage detected



=== TARGET LEAKAGE CHECK ===
No target leakage detected in feature columns


In [19]:
# Compare distributions before/after cleaning
print("\n=== DISTRIBUTION COMPARISON ===")
important_cols = ['RIDAGEYR', 'BMXBMI', 'BMXWT', 'BPXSY1', 'BPXDI1']
dist_comparison = compare_distributions(df_raw, df_minimal, important_cols)
dist_comparison[['variable', 'mean_before', 'mean_after', 'mean_change_pct', 'missing_before', 'missing_after']]

2026-01-29 13:40:06,222 - INFO - No large distribution changes detected



=== DISTRIBUTION COMPARISON ===


,variable,mean_before,mean_after,mean_change_pct,missing_before,missing_after
0,RIDAGEYR,49.200034,48.476329,-1.470945,0,0
1,BMXBMI,29.523059,29.523059,0.000000,751,751
2,BMXWT,81.914236,81.600746,-0.382705,737,737
3,BPXSY1,125.705952,125.705952,0.000000,1541,1541
4,BPXDI1,70.659595,70.659595,0.000000,1541,1541


In [20]:
# Final validation
print("\n=== FINAL VALIDATION ===")
minimal_valid = validate_cleaned_data(df_minimal, expected_row_count=len(df_raw))
full_valid = validate_cleaned_data(df_full, expected_row_count=len(df_raw))

print(f"\nMinimal imputed data valid: {minimal_valid}")
print(f"Full imputed data valid: {full_valid}")

2026-01-29 13:40:06,227 - INFO - Cleaned data validation PASSED


2026-01-29 13:40:06,228 - INFO - Cleaned data validation PASSED



=== FINAL VALIDATION ===

Minimal imputed data valid: True
Full imputed data valid: True


---
## 6. Save Outputs <a name="6-save-outputs"></a>

In [21]:
# Save minimal imputed version (for LightGBM)
minimal_path = project_root / 'data' / 'interim' / 'cleaned_minimal_impute.parquet'
df_minimal.to_parquet(minimal_path, index=False)
print(f"Saved minimal imputed data to: {minimal_path}")
print(f"  Shape: {df_minimal.shape}")

Saved minimal imputed data to: /Users/timhall/Library/Mobile Documents/com~apple~CloudDocs/Projects/diabetes_prediction_project/data/interim/cleaned_minimal_impute.parquet
  Shape: (11723, 1562)


In [22]:
# Save full imputed version (for LogReg/NN)
full_path = project_root / 'data' / 'interim' / 'cleaned_full_impute.parquet'
df_full.to_parquet(full_path, index=False)
print(f"Saved full imputed data to: {full_path}")
print(f"  Shape: {df_full.shape}")

Saved full imputed data to: /Users/timhall/Library/Mobile Documents/com~apple~CloudDocs/Projects/diabetes_prediction_project/data/interim/cleaned_full_impute.parquet
  Shape: (11723, 1562)


In [23]:
# Generate and save cleaning report
cleaning_log = {
    'steps': {
        'recode_special_values': recode_log,
        'missing_flags': {
            'threshold': MISSING_FLAG_THRESHOLD,
            'n_flags_created': len(flag_columns),
            'flag_columns': flag_columns
        },
        'minimal_imputation': {
            'threshold': IMPUTE_THRESHOLD,
            'columns_imputed': minimal_impute_log
        },
        'full_imputation': {
            'columns_imputed': full_impute_log
        }
    }
}

report_path = project_root / 'data' / 'interim' / 'cleaning_report.json'
report = generate_cleaning_report(df_raw, df_minimal, cleaning_log, report_path)

print(f"\nCleaning report saved to: {report_path}")

2026-01-29 13:40:06,718 - INFO - 
Running validation checks...


2026-01-29 13:40:06,722 - WARNING - Found 215 values out of expected ranges


2026-01-29 13:40:06,723 - WARNING -   BMXBMI: 2 out of range


2026-01-29 13:40:06,723 - WARNING -   LBXGLU: 1 out of range


2026-01-29 13:40:06,723 - WARNING -   BPXDI1: 70 out of range


...

2026-01-29 13:40:06,728 - WARNING -   waist_lt_height: 9 violations


2026-01-29 13:40:06,728 - WARNING -   diabetes_meds_implies_diabetes: 5 violations


2026-01-29 13:40:06,732 - INFO - No large distribution changes detected


2026-01-29 13:40:06,735 - INFO - Cleaning report saved to: /Users/timhall/Library/Mobile Documents/com~apple~CloudDocs/Projects/diabetes_prediction_project/data/interim/cleaning_report.json



Cleaning report saved to: /Users/timhall/Library/Mobile Documents/com~apple~CloudDocs/Projects/diabetes_prediction_project/data/interim/cleaning_report.json


In [24]:
# Display validation summary
print("\n=== VALIDATION SUMMARY ===")
print(f"Row count preserved: {report['validation_summary']['row_count_preserved']}")
print(f"Range violations: {report['validation_summary']['n_range_violations']}")
print(f"Logic violations: {report['validation_summary']['n_logic_violations']}")
print(f"Overall validation: {'PASSED' if report['validation_summary']['passed'] else 'FAILED'}")


=== VALIDATION SUMMARY ===
Row count preserved: True
Range violations: 215
Logic violations: 697
Overall validation: FAILED


---
## 7. Summary <a name="7-summary"></a>

In [25]:
print("="*60)
print("PHASE 3 SUMMARY: Data Cleaning & Preprocessing")
print("="*60)

print("\n1. SPECIAL VALUE RECODING")
print(f"   'Refused' (7/77/777...) → -7: {sum(recode_log['refused'].values()):,} values")
print(f"   'Don't know' (9/99/999...) → -9: {sum(recode_log['dont_know'].values()):,} values")

print("\n2. MISSING FLAGS")
print(f"   Threshold: {MISSING_FLAG_THRESHOLD*100:.0f}%")
print(f"   Flags created: {len(flag_columns)}")

print("\n3. IMPUTATION")
print(f"   Minimal (LightGBM): {len(minimal_impute_log)} columns imputed (<{IMPUTE_THRESHOLD*100:.0f}% missing)")
print(f"   Full (LogReg/NN): {len(full_impute_log)} columns imputed (<={MAX_MISSING_RATE*100:.0f}% missing)")
print(f"   Skipped (>{MAX_MISSING_RATE*100:.0f}% missing): {len(high_missing_cols)} columns")
print(f"   Target columns preserved: {', '.join([c for c in TARGET_RELATED_COLS if c in df_raw.columns])}")

print("\n4. VALIDATION")
print(f"   Range violations: {report['validation_summary']['n_range_violations']}")
print(f"   Logic violations: {report['validation_summary']['n_logic_violations']}")
print(f"   Target leakage: {'None detected' if not leaked_cols else leaked_cols}")

print("\n5. OUTPUT FILES")
print(f"   cleaned_minimal_impute.parquet: {df_minimal.shape}")
print(f"   cleaned_full_impute.parquet: {df_full.shape}")
print(f"   cleaning_report.json: Saved")

print("\n6. NEXT STEPS (Phase 4)")
print("   - Feature engineering")
print("   - Create derived features (BMI categories, BP averages, etc.)")
print("   - Define feature sets (with_labs, without_labs)")

PHASE 3 SUMMARY: Data Cleaning & Preprocessing

1. SPECIAL VALUE RECODING
   'Refused' (7/77/777...) → -7: 21,364 values
   'Don't know' (9/99/999...) → -9: 33,388 values

2. MISSING FLAGS
   Threshold: 5%
   Flags created: 730

3. IMPUTATION
   Minimal (LightGBM): 37 columns imputed (<5% missing)
   Full (LogReg/NN): 288 columns imputed (<=50% missing)
   Skipped (>50% missing): 479 columns
   Target columns preserved: DIQ070, LBXGLU, LBXGH, DIABETES_STATUS, DIQ010, DIQ050

4. VALIDATION
   Range violations: 215
   Logic violations: 697
   Target leakage: None detected

5. OUTPUT FILES
   cleaned_minimal_impute.parquet: (11723, 1562)
   cleaned_full_impute.parquet: (11723, 1562)
   cleaning_report.json: Saved

6. NEXT STEPS (Phase 4)
   - Feature engineering
   - Create derived features (BMI categories, BP averages, etc.)
   - Define feature sets (with_labs, without_labs)
